# Deep Learning for Computer Vision with TensorFlow 2.0

Table of contents:
[Lab 0](https://colab.research.google.com/github/edge-ai-vision/dlcvtf2/blob/master/00_test_install.ipynb) | 
[Lab 1](https://colab.research.google.com/github/edge-ai-vision/dlcvtf2/blob/master/01_linear_regression.ipynb) | 
[Lab 2](https://colab.research.google.com/github/edge-ai-vision/dlcvtf2/blob/master/02_tensorflow_logistic_regression.ipynb) | 
[Lab 3a](https://colab.research.google.com/github/edge-ai-vision/dlcvtf2/blob/master/03a_tensorflow_deep_network.ipynb) | 
[Lab 3b](https://colab.research.google.com/github/edge-ai-vision/dlcvtf2/blob/master/03b_deep_mnist_visualize.ipynb) | 
[Lab 4](https://colab.research.google.com/github/edge-ai-vision/dlcvtf2/blob/master/04_mnist_cnn.ipynb) | 
[Lab 5](https://colab.research.google.com/github/edge-ai-vision/dlcvtf2/blob/master/05_data_prep.ipynb) | 
[Lab 6](https://colab.research.google.com/github/edge-ai-vision/dlcvtf2/blob/master/06_transfer_learning.ipynb) | 

# Lab 3B: Deep MNIST with Visualization

If you ever want to start over just go to the menu at the top and select Runtime ->  Restart runtime.  (Please do not use Runtime -> Reset all runtimes.)

In this exercise you will use the deep model used in the last exercise but be we are going to closely monitor the training process.  

The training process is setup similar to the previous example, but there is an issue with one of the cells. It will be pointed out, but run it as provided and update the model as instructed later to make it work properly. 


In [0]:
# Cell 3b.1
%tensorflow_version 2.x
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

import numpy as np
%pylab inline

print ('cell finished')

The next cell sets up the default settings for our training variables once again.  

The default starting value of the `LEARNING_RATE` variable is wrong on purpose. This is to let us see what happens in the training process when something goes wrong. Run through the process once to see the abysmal values, then come back to this cell and change `LEARNING_RATE` to .001. 

Run the process again to see that it is working properly.

In [0]:
# Cell 3b.2

BATCH_SIZE=100

HIDDEN_SIZE = 300
LEARNING_RATE = 0.5 # This is much too large, but run with it initally and change it later as instructed

NUM_CLASSES = 10
IMG_HT = 28
IMG_W = 28

print ('cell finished')

The next cell reads in the MNIST dataset as before and sets up our model parameters. 

In [0]:
# Cell 3b.3

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
# Numpy defaults to dtype=float64; TF defaults to float32. Stick with float32.
x_train, x_test = x_train / np.float32(255), x_test / np.float32(255)
y_train, y_test = y_train.astype(np.int64), y_test.astype(np.int64)

train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).repeat()
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test))

# DON'T CHANGE THESE VARIABLES OR YOU WILL BREAK THINGS
NUM_PIXELS = 28 * 28
NUM_CLASSES = 10

print ('cell finished')

This is the same code to setup our placeholders

In [0]:
# Cell 3b.4

train_ds = train_ds.shuffle(60000).batch(BATCH_SIZE)
test_ds = test_ds.batch(BATCH_SIZE)

print ('cell finished')


This cell is the same model as before. It uses the constants from cell 2 to size the network

In [0]:
# Cell 3b.5

def model():
    model = keras.Sequential([
    layers.Flatten(input_shape=[IMG_HT,IMG_W]),
    layers.Dense(HIDDEN_SIZE, activation='relu'),
    layers.Dense(NUM_CLASSES, activation='softmax')
  ])
    return model

tb_callback = tf.keras.callbacks.TensorBoard(log_dir='log_dir')

print ('cell finished')


Creating the model and displaying the summary as before 

In [0]:
# Cell 3b.6

mnist_model = model()

mnist_model.summary()

print ('cell finished')


This cell creates an optimizer directly using the `LEARNING_RATE` constant from cell 2

In [0]:
# Cell 3b.7

optimizer = tf.keras.optimizers.Adam(LEARNING_RATE)


The model is compiled and instead of specifying the built-in optimizer with default values the optimizer object created above is used as the optimizer argument.

In [0]:
# cell 3b.8

mnist_model.compile(optimizer=optimizer, 
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

print ('cell finished')


Now the model will be fit to the data using our standard technique. However the first time through the loss will be high and the accuracy low. After the learning process is fixed, the second time through should show much better results

In [0]:
# cell 3b.9

history = mnist_model.fit(
  train_ds,
  epochs=20,  
  steps_per_epoch=600,verbose=2,callbacks=[tb_callback])

print ('cell finished')

When the fit method is run data about the run is returned. In our example this data is gathered in an object named `history` which was provided as the output variable from the fit process. 

The data is a dictionary object whose contents are based on the metrics requested for the training process. In this example the accuracy `acc` and loss `loss` were captured and can be plot over each epoch of the training. 

In [0]:
# Cell 3b.10

#pylab.plot(sqft_train, price_train, 'b.')

pylab.plot(history.history['accuracy'],'b')
pylab.title('Model accuracy')
pylab.ylabel('Accuracy')
pylab.xlabel('Epoch')
pylab.legend(['Train'], loc='upper left')
pylab.show()

# Plot training & validation loss values
pylab.plot(history.history['loss'], 'r')
#plt.plot(history.history['val_loss'])
pylab.title('Model loss')
pylab.ylabel('Loss')
pylab.xlabel('Epoch')
pylab.legend(['Train'], loc='upper left')
pylab.show()

print ('cell finished')

The plots above were for the accuracy and loss over time for each epoch. Below the accuracy of model using the testing data will be evaluated using the evaluate method. A history object is once again returned but since the evaluation only runs for one epoch, the history object will return one value. 

In [0]:
# Cell 3b.11

history=mnist_model.evaluate(
    test_ds, steps=100)


print ('\n cell finished')

This cell just reformats the data for ease of understanding.

In [0]:
# Cell 3b.12

print ('loss = {} accuracy = {}'.format (history[0], history[1]))

print ('cell finished')


As you can see the accuracy is terrible. Something is obviously wrong with the learning process. While there are lots of things that can go wrong, such as bad data, bad data pipelining, etc. hyperparameter settings can also affect the output. In this case the `LEARNING_RATE` is much too high for the first pass. 

Do the following:

1. Restart the notebook - from the Runtime menu at the top of the page select **Restart runtime** (note: do not select "Reset all runtimes")
2. Change LEARNING_RATE - go to cell 2 and modify LEARNING_RATE to 0.001
3. Re-run all cells - start at cell 1 and run every cell in the notebook to cell 12




## If you have time
You can experiment with other learning rates to see what happens